# TODO
* check if lemma actually occurs in WSD corpora
* tf idf based measure to determine how domain specific a lemma is
* ignore monosemous words (monosemous WRT to lemma, pos combination in WordNet)
* think about preferred variance and ambiguity

In [1]:
from nltk.corpus import wordnet as wn
import pickle
import operator
import os

In [2]:
with open('output/lemma2freq_titles_content.pickle', 'rb') as infile:
    lemma_freqs = pickle.load(infile)

In [3]:
output_path = 'output/wsd_noun_annotations.pickle'

In [4]:
if os.path.exists(output_path):
    with open(output_path, 'rb') as infile:
        annotations = pickle.load(infile)
else:
    annotations = dict()

In [5]:
min_polysemy = 2
pos = 'n'

In [6]:
for lemma, freq in sorted(lemma_freqs['NOUN'].items(),
                          key=operator.itemgetter(1),
                          reverse=True):
    
    # continue if already annotated
    if (lemma, pos) in annotations:
        continue
    
    iden2eng = dict()
    synsets = wn.synsets(lemma, pos=pos)
    if len(synsets) >= min_polysemy:
        for identifier, synset in enumerate(synsets, 1):
            
            offset     = synset.offset()
            len_offset = len(str(offset)) 
            eng_def = "eng-30-{offset}-{pos}".format(offset="0"*(8-len_offset)+str(offset),
                                                    pos=synset.pos())
            iden2eng[identifier] = identifier, eng_def
            
            print('LEMMA: %s' % lemma)
            print()
            print('IDENTIFIER: %s' % identifier)
            print()
            print(synset, synset.definition())
            print(synset.examples())
            print(eng_def)
            print()
            for hypernym in synset.hypernyms():
                print('HYPERNYM', hypernym, hypernym.definition())
                
        annotation_not_complete = True
        while annotation_not_complete:
            annotation = input('dominant meaning identifier: ')

            try:
                annotation_int = int(annotation)
            except ValueError:
                continue

            annotations[(lemma, pos)] = iden2eng[annotation_int]
            
            # save
            with open(output_path, 'wb') as outfile:
                pickle.dump(annotations, outfile)
                
            annotation_not_complete = False

LEMMA: block

IDENTIFIER: 1

Synset('block.n.01') a solid piece of something (usually having flat rectangular sides)
['the pyramids were built with large stone blocks']
eng-30-02852523-n

HYPERNYM Synset('artifact.n.01') a man-made object taken as a whole
LEMMA: block

IDENTIFIER: 2

Synset('block.n.02') a rectangular area in a city surrounded by streets and usually containing several buildings
['he lives in the next block']
eng-30-08642145-n

HYPERNYM Synset('area.n.01') a particular geographical region of indefinite boundary (usually serving some special purpose or distinguished by its people or culture or geography)
LEMMA: block

IDENTIFIER: 3

Synset('block.n.03') a three-dimensional shape with six square or rectangular sides
[]
eng-30-13914608-n

HYPERNYM Synset('solid.n.03') a three-dimensional shape
LEMMA: block

IDENTIFIER: 4

Synset('block.n.04') a number or quantity of related things dealt with as a unit
['he reserved a large block of seats', "he held a large block of the com

KeyboardInterrupt: 

In [7]:
print(len(annotations))

10
